<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/RayleighDistillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [2]:
from dotmap import DotMap
import jax
import jax.numpy as jnp
from jax.config import config
from jax.experimental.host_callback import id_print
config.update("jax_enable_x64", True) #JAX default is 32 bit - enable 64 bit - double precision

from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'

from scipy.integrate import solve_ivp

from tools.tree_array_transform2 import VSC, Comp, Range, RangeArray
import tools.che as che
R=8.314 # J/(mol K)

In [3]:
p = che.Props(['Methanol', 'Isopropanol'])

In [4]:
def bubble_T(c):
    return jnp.sum(c.x * p.Pvap(c.T))-c.P

c = DotMap()
c.T = Range(300, 273.15, 400.)
P=101325

def solve_bubble_T(x1, P):
  c.x = jnp.array([x1, 1-x1])
  c.P = P
  vsc = VSC(c, bubble_T)
  vsc.solve(verbosity=0)
  return (vsc.v.T)

In [5]:
qin_Hvap = 1.
L0 = 10.
tend = 9.

def dx1(t, x1):
  T = solve_bubble_T(x1, P)
  y1 = x1*p.Pvap(T)[0]/P
  return (y1-x1)*qin_Hvap/(L0 - qin_Hvap*t)

res = solve_ivp(dx1, (0, tend), [0.5], dense_output=True)

In [6]:
tplot = jnp.linspace(0,tend,10)
x1_plot = res.sol(tplot)[0]
temp_plot = [solve_bubble_T(xx,P) for xx in x1_plot]
fig = make_subplots(rows=1,cols=2)
fig.add_scatter(x=tplot, y=x1_plot, mode='lines', name='x1',row=1, col=1)
fig.add_scatter(x=tplot, y=temp_plot, mode='lines', name='T',row=1, col=2)
fig.update_layout(width=800, height=400)